In [1]:
from vivit import run_experiment, test_files, classes
from data import get_video_label, VideoDataGenerator, labels_df, files, labels
import synthetic
import scipy.stats as stats
from sklearn.model_selection import train_test_split
import numpy as np
import keras
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
classes = list(labels_df.groupby('label').size().sort_values(ascending=False)[:10].index)
good_classes = ['book', "drink", "go", "chair", "who", "clothes", "yes", "year", "thin", "fine"]
test_files = [f for f in files if labels[int(os.path.basename(f).split('.')[0])] in good_classes]

In [3]:
base = "../data/videos"
for video, f in enumerate(test_files):
    name = os.path.basename(f)
    if not os.path.exists(f"{base}/flipped/{name}"):
        synthetic.flip_video(f"{base}/original/{name}", f"{base}/flipped/{name}")
    if not os.path.exists(f"{base}/rotated_90/{name}"):
        synthetic.rotate_video(f"{base}/original/{name}", f"{base}/rotated_90/{name}", 90)
    if not os.path.exists(f"{base}/rotated_270/{name}"):
        synthetic.rotate_video(f"{base}/original/{name}", f"{base}/rotated_270/{name}", 270)
    if not os.path.exists(f"{base}/noised/{name}"):
        synthetic.noise_video(f"{base}/original/{name}", f"{base}/noised/{name}")
    print(f"{video+1}/{len(test_files)}", end = '\r')

In [4]:
all_files = []
for folder in ["original", "flipped", "rotated_90", "rotated_270", "noised"]:
    all_files += [f"{base}/{folder}/{os.path.basename(f)}" for f in test_files]
y = [get_video_label(f) for f in all_files]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(all_files, y, test_size=0.2, random_state=42, stratify=y)
train, validation = VideoDataGenerator(X_train), VideoDataGenerator(X_test)

In [ ]:
model = run_experiment(name = "high-epoch", data = train, validation = validation, epochs = 250)

In [9]:
model.save("../models/END-high-epoch.keras")

In [16]:
model = keras.models.load_model("../models/high-epoch.keras")
model.evaluate(validation, verbose = 1)

9/9 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.7998 - loss: 0.9216 - top-5-accuracy: 0.9692


[0.9110530018806458, 0.7886792421340942, 0.9735848903656006]